# File conversion
## SAM to BAM (and vice versa)

## BAM to CRAM
We will convert a yeast BAM file to CRAM. In the data directory, there is a BAM file called yeast.bam that was created from S. cerevisiae Illumina sequencing data.

 Can you convert the BAM file to a CRAM file called yeast.cram using the samtools view command? Note the reference genome is stored in the file Saccharomyces_cerevisiae.EF4.68.dna.toplevel.fa
 Since CRAM files use reference based compression, we expect the CRAM file to be smaller than the BAM file. What is the size of the CRAM file?

 Is your CRAM file smaller than the original BAM file?
## FASTQ to SAM/BAM (and vice versa)
## FASTQ to CRAM (and vice versa)
## VCV to BCF (and vice versa)